In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # Must be before importing keras!
import sys
sys.path.insert(0,"/home/kal/CTCF/modules/")
sys.path.insert(0,"/home/kal/CTCF/mass_CTCF/modules/")
import tf_memory_limit
#import matplotlib; matplotlib.use('Agg')  # Must be before importing matplotlib.pyplot or before importing pylab!
from keras import backend as K
import tensorflow as tf
import numpy as np
import ctcfgen
import eval_convnet_ctcf
import ucscgenome
from convnet import Bias
from keras.models import load_model, Model
from keras.layers import Input, Activation
import viz_sequence
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from scipy.signal import correlate2d
from sklearn.metrics import precision_recall_curve, roc_curve
from scipy.special import logit

colors = {0:'crimson', 1:'steelblue', 2:'goldenrod', 3:'forestgreen'}

batch_size=32
input_window=256

meme_path = '/home/kal/data/CTCF.meme'
gen_path='/home/kal/data/ctcfgen_data.hdf5'
motif_bed_path = '/home/kal/data/1000genomes_motif_SNVs.bed'
all_bed_path = '/home/kal/data/1000genomes_all_SNVs_motif.bed'

# load the model
model_path = '/home/kal/CTCF/mass_CTCF/output/sigmoid/20170919_150754/20170919_150754_weights_3_248_0.91.hdf5'
model2_path = '/home/kal/CTCF/mass_CTCF/output/tanh/20170919_154126/20170919_154126_weights_3_241_0.90.hdf5'
#model_path = '/home/kal/CTCF/mass_CTCF/eric_output/20170731_014309/20170731_014309_weights_3_243_0.91.hdf5'
#model2_path = '/home/kal/CTCF/mass_CTCF/output/sigmoid/20170919_150754/20170919_150754_weights_3_248_0.91.hdf5'

model = load_model(model_path, custom_objects={'Bias':Bias})
use_act1 = True
get_act = K.function([model.input, K.learning_phase()], [layer_dict['bias'].output])
model2 = load_model(model2_path, custom_objects={'Bias':Bias})
use_act2 = True

# load the generator
gen = ctcfgen.CTCFGeneratorhdf5(gen_path)

#load the genome
genome = ucscgenome.Genome('/home/kal/.ucscgenome/hg19.2bit')

# get the symbolic outputs of each "key" layer.
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_dict2 = dict([(layer.name, layer) for layer in model2.layers])
print(layer_dict.keys())

In [ ]:
def softmax(batch):
    norm_batch = []
    for seq in batch:
        norm_batch.append([_softmax(base) for base in seq])
    return np.asarray(norm_batch)
    
def _softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only differenc

In [ ]:
# process the 1000 genomes data
snvs = pd.read_table(all_bed_path, header=None)
snvs.columns = 'chr varstart varend TF ref A C G T motif_count'.split()
snvs['motif'] = snvs['motif_count'] > 0
snvs['start'] = snvs['varstart'] - 128
snvs['end'] = snvs['varstart'] + 128
order = {'A':0, 'C':1, 'G':2, 'T':3}
rorder = ['A', 'C', 'G', 'T']
ctcf_snvs = snvs[snvs.TF.apply(lambda x: 'CTCF' in x)].copy()
ratios=list()
alts=list()
for idx, row in ctcf_snvs.iterrows():
    counts = [row['A'], row['C'], row['G'], row['T']]
    ref_count = counts[order[row['ref']]]
    counts[order[row['ref']]] = 0
    alt_count = max(counts)
    alt = rorder[counts.index(alt_count)]
    ratio = alt_count/(ref_count + alt_count)
    alts.append(alt)
    ratios.append(ratio)
    
ctcf_snvs['alt'] = np.asarray(alts)
ctcf_snvs['allelicRatio'] = np.asarray(ratios)
ctcf_snvs['mut'] = (ctcf_snvs.allelicRatio > .5)


# load the bed
#snvs = pd.read_table(motif_bed_path, header=None, sep=',')
#snvs.columns = 'chr,varstart,varend,ref,alt,ref_freq,alt_freq,diff_freq,motif,TF,indiv_TF,allelicRatio'.split(',')
#snvs['start'] = snvs['varstart'] - 128
#snvs['end'] = snvs['varstart'] + 128

#ctcf_snvs = snvs[snvs.TF.apply(lambda x: 'CTCF' in x)].copy()


In [ ]:
# add annotations to the peaks
for index, row in ctcf_snvs.iterrows():
    # get the sequence
    center = (row.start + row.end) // 2
    if center < 128:
        center = 128
    orig_seq = ctcfgen.encode(np.fromstring(genome[row.chr][center-128:center+128].lower(), dtype=np.uint8))
    new_seq = orig_seq.copy()
    new_seq[128:129] = ctcfgen.encode_string(row['alt'].lower())
    
    # add the model scores
    ctcf_snvs.set_value(index,'oml_score', get_act([np.asarray([orig_seq]*32),0])[0][0])
    ctcf_snvs.set_value(index,'nml_score', get_act([np.asarray([new_seq]*32),0])[0][0])

    # add the ctcf pwm scores
    score, pwm, meme, meme_index, position = ctcfgen.get_pwm([orig_seq], get_everything=True)
    ctcf_snvs.set_value(index,'octcf', score)
    score, pwm, meme, meme_index, position = ctcfgen.get_pwm([new_seq], get_everything=True, meme=meme, position=position)
    ctcf_snvs.set_value(index, 'nctcf', score)
    ctcf_snvs.set_value(index,'ctcf_orentation', (meme_index>1))
    ctcf_snvs.set_value(index,'ctcf_position', position)

    
    # add the mystery pwm scores
    score, pwm, meme, meme_index, position = ctcfgen.get_pwm([orig_seq], get_everything=True, meme_library='mystery_memes')
    ctcf_snvs.set_value(index,'omystery', score)
    score, pwm, meme, meme_index, position = ctcfgen.get_pwm([new_seq], get_everything=True, meme=meme, position=position, meme_library='mystery_memes')
    ctcf_snvs.set_value(index, 'nmystery', score)
    ctcf_snvs.set_value(index,'mystery_orentation', (meme_index>0))
    ctcf_snvs.set_value(index,'mystery_position', position)
    
ctcf_snvs['distance'] = abs(ctcf_snvs['mystery_position'] - ctcf_snvs['ctcf_position'])

In [ ]:
orig_preds = list()
new_preds = list()
o_preds = list()
n_preds = list()
orig_pwms = list()
new_pwms = list()
in_pwm = list()
mo_pwms=list()
mn_pwms=list()

if use_act1:
    get_act = K.function([model.input, K.learning_phase()], [layer_dict['bias'].output])
else:
    get_act = K.function([model.input, K.learning_phase()], [model.output])
    
if use_act2:
    get_act2 = K.function([model2.input, K.learning_phase()], [layer_dict2['bias'].output])
else:
    get_act2 = K.function([model2.input, K.learning_phase()], [model2.output])

for index, row in ctcf_snvs.iterrows():
    # get mutant version of the sequence
    orig_seq = ctcfgen.encode(np.fromstring(genome[row.chr][row.start:row.end].lower(), dtype=np.uint8))
    new_seq = orig_seq.copy()
    new_seq[128:129] = ctcfgen.encode_string(row['alt'].lower())
         
    # get predictions
    orig_pred = get_act([np.asarray([orig_seq]*32),0])[0][0]
    new_pred = get_act([np.asarray([new_seq]*32),0])[0][0]
    
    #o_pred = get_act2([np.asarray([np.append(orig_seq, np.asarray([10]*256)).reshape(-1, 5)]*32),0])[0][0]
    #n_pred = get_act2([np.asarray([np.append(new_seq, np.asarray([10]*256)).reshape(-1, 5)]*32),0])[0][0]
    o_pred = get_act2([np.asarray([orig_seq]*32),0])[0][0]
    n_pred = get_act2([np.asarray([new_seq]*32),0])[0][0]
    
    # get scores    
    fseq, orig_score = ctcfgen.get_pwm([orig_seq], get_score=True)
    fseq, new_score = ctcfgen.get_pwm([new_seq], get_score=True)   
    
    fseq, morig_score = ctcfgen.get_pwm([orig_seq], get_score=True, meme_library='mystery_memes')
    fseq, mnew_score = ctcfgen.get_pwm([new_seq], get_score=True, meme_library='mystery_memes')  
     
    in_pwm.append(np.amax(fseq[118:138])>0)
    
    orig_preds.append(orig_pred)
    new_preds.append(new_pred)
    orig_pwms.append(orig_score)    
    new_pwms.append(new_score)
    mo_pwms.append(morig_score)
    mn_pwms.append(mnew_score)
    o_preds.append(o_pred)
    n_preds.append(n_pred)
    
ctcf_snvs['refpred'] = np.asarray(orig_preds)
ctcf_snvs['altpred'] = np.asarray(new_preds)

ctcf_snvs['odiffpred'] = np.asarray(n_preds) - np.asarray(o_preds)

ctcf_snvs['refscore'] = np.asarray(orig_pwms)
ctcf_snvs['altscore'] = np.asarray(new_pwms)

ctcf_snvs['mrefscore'] = np.asarray(mo_pwms)
ctcf_snvs['maltscore'] = np.asarray(mn_pwms)

ctcf_snvs['diffpred'] = ctcf_snvs['altpred']-ctcf_snvs['refpred']
ctcf_snvs['diffscore'] = ctcf_snvs['altscore']-ctcf_snvs['refscore']
ctcf_snvs['diffmscore'] = ctcf_snvs['maltscore']-ctcf_snvs['mrefscore']

ctcf_snvs['pwm_motif'] = np.asarray(in_pwm)

In [ ]:
filtered = ctcf_snvs
#filtered = ctcf_snvs[~(ctcf_snvs['pwm_motif'])]
#filtered = filtered[(filtered['diffpred']!=0)]
ratios = filtered['allelicRatio'].tolist()
pwms = filtered['diffscore'].tolist()
mls = filtered['diffpred'].tolist()
omls = filtered['odiffpred'].tolist()


plt.hexbin(ratios, mls, gridsize=50, bins='log', cmap='plasma')
plt.title('Single Nucleotide Varients in CTCF')
plt.xlabel('Ratio Alternate/Total Reads')
plt.ylabel('Difference in Sigmoid Model Score')
plt.show()

plt.hexbin(ratios, pwms, gridsize=50, bins='log', cmap='plasma')
plt.title('Single Nucleotide Varients in CTCF')
plt.xlabel('Ratio Alternate/Total Reads')
plt.ylabel('Difference in PWM Score')
plt.show()

plt.hexbin(ratios, omls, gridsize=50, bins='log', cmap='plasma')
plt.title('Single Nucleotide Varients in CTCF')
plt.xlabel('Ratio Alternate/Total Reads')
plt.ylabel('Difference in Tanh Model Score')
plt.show()

plt.hexbin(omls, mls, gridsize=50, bins='log', cmap='plasma')
plt.title('Single Nucleotide Varients in CTCF')
plt.xlabel('Difference in Tanh model Score')
plt.ylabel('Difference in Sigmoid model Score')
plt.show()

In [ ]:
# make histograms

plt.hist(filtered[(filtered['mut'])]['diffscore'], 30, alpha=.5, label='Positives')
plt.hist(filtered[~(filtered['mut'])]['diffscore'], 30, alpha=.5, label='Negatives')
plt.title('Position Weight Matrix Prediction for CTCF binding')
plt.xlabel('Position Weight Matrix Correlation')
plt.legend()
plt.show()


plt.hist(filtered[(filtered['mut'])]['diffpred'], 30, alpha=.5, label='Positives')
plt.hist(filtered[~(filtered['mut'])]['diffpred'], 30, alpha=.5, label='Negatives')
plt.title('Deep Learning Sigmoid Model for CTCF binding')
plt.xlabel('Deep Learning Score')
plt.legend()
plt.show()

plt.hist(filtered[(filtered['mut'])]['odiffpred'], 30, alpha=.5, label='Positives')
plt.hist(filtered[~(filtered['mut'])]['odiffpred'], 30, alpha=.5, label='Negatives')
plt.title('Deep Learning Tanh Model for CTCF binding')
plt.xlabel('Deep Learning Score')
plt.legend()
plt.show()

In [ ]:
deep_fpr, deep_tpr, thresholds = roc_curve(filtered['mut'].tolist(),
                                                                 mls, pos_label=1)

pwm_fpr, pwm_tpr, thresholds = roc_curve(filtered['mut'].tolist(),
                                                                 pwms, pos_label=1)

str_fpr, str_tpr, thresholds = roc_curve(filtered['mut'].tolist(),
                                                                 omls, pos_label=1)

plt.plot(deep_fpr, deep_tpr, label='Deep Learning')
plt.plot(pwm_fpr, pwm_tpr, label = 'Position-Weight Matrix')
plt.plot(str_fpr, str_tpr, label = 'Alternate Model')
plt.legend()
plt.title('ROC Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.show()

deep_precision, deep_recall, thresholds = precision_recall_curve(filtered['mut'].tolist(),
                                                                 mls, pos_label=1)

pwm_precision, pwm_recall, thresholds = precision_recall_curve(filtered['mut'].tolist(),
                                                                 pwms, pos_label=1)

str_precision, str_recall, thresholds = precision_recall_curve(filtered['mut'].tolist(),
                                                                 omls, pos_label=1)

plt.plot(deep_recall, deep_precision, label='Deep Learning')
plt.plot(pwm_recall, pwm_precision, label = 'Position-Weight Matrix')
plt.plot(str_recall, str_precision, label='Alternate Model')

plt.legend()
plt.title('Precision-Recall Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.show()

prop_pos = len(ctcf_snvs.loc[ctcf_snvs['mut'] == 1])/len(ctcf_snvs)
print('Proportion positive:' + str(prop_pos))
deep_precision_gain = [(x-prop_pos)/((1-prop_pos)*x) for x in deep_precision]
deep_recall_gain = [(x-prop_pos)/((1-prop_pos)*x) for x in deep_recall]

pwm_precision_gain = [(x-prop_pos)/((1-prop_pos)*x) for x in pwm_precision]
pwm_recall_gain = [(x-prop_pos)/((1-prop_pos)*x) for x in pwm_recall]

str_precision_gain = [(x-prop_pos)/((1-prop_pos)*x) for x in str_precision]
str_recall_gain = [(x-prop_pos)/((1-prop_pos)*x) for x in str_recall]

plt.plot(deep_recall_gain, deep_precision_gain, label='Deep Learning')
plt.plot(pwm_recall_gain, pwm_precision_gain, label = 'Position-Weight Matrix')
plt.plot(str_recall_gain, str_precision_gain, label = 'Alternate Model')

plt.legend()
plt.title('Precision-Recall Gain Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.show()

In [ ]:
def act_mutagenisis(seq, model=model):
    get_act = K.function([model.input, K.learning_phase()], [layer_dict['bias'].output])

    # do a mutant scan
    mutant_window=1

    # get a mutant batch generator
    mutant_gen = ctcfgen.ngram_mutant_gen(seq, n=mutant_window)
    g = ctcfgen.filled_batch(mutant_gen, batch_size=batch_size)
    
    # base importances as large-step gradients
    # score with base there - average of scores without base there
    mutant_preds = list()
    for batch in g:
        mutant_preds.append(get_act([batch, 0])[0])

    return np.asarray(mutant_preds).reshape((-1, 4))  

In [ ]:
def get_importance(seq, my_model=model, use_act=True, start=0, end=256):
    my_layer_dict = dict([(layer.name, layer) for layer in my_model.layers])
    get_act = K.function([my_model.input, K.learning_phase()], [my_layer_dict['bias'].output])

    pred = my_model.predict(np.asarray([seq]*32))[0][0]
    if use_act:
        score = get_act([[seq]*32, 0])[0][0][0]
    else:
        score = pred
    print('Prediction ' + str(pred))

    if use_act:
        mutant_preds = act_mutagenisis(seq, model=model)
    else:
        mutant_preds = mutagenisis(seq, model=model)
      
    diffs = mutant_preds - score
   
    all_diffs = list()
    for base_seq, base_preds in zip(seq, mutant_preds):
        this_base = list()
        for idx in range(4):
            this_base.append(base_preds[idx] - np.average(base_preds))
        all_diffs.append(list(this_base))

    all_diffs=np.asarray(all_diffs)

    score_diff = list()
    for base_seq, base_preds in zip(seq, mutant_preds):
        idx = np.where(base_seq)
        score_diff.append(base_preds[idx] - np.average(base_preds))
      
    score_diff = (seq * score_diff)
    
    print('prediciton difference')
    viz_sequence.plot_weights(all_diffs[start:end], colors=colors)

    print('Masked average prediciton difference')
    viz_sequence.plot_weights(score_diff[start:end], colors=colors)

    #print('Softmax average prediction difference')
    #viz_sequence.plot_icweights(softmax([all_diffs[start:end]])[0], colors=colors)

In [ ]:
# find seqs of interest
for index, row in ctcf_snvs.iterrows():
    if (row.diffscore == 0) and ((abs(row.diffpred) > 1) or (abs(row.odiffpred) > 1)) and (((row.odiffpred>0) == (row.allelicRatio>.5)) or ((row.diffpred>0) == (row.allelicRatio>.5))) :
    #if abs(row.odiffpred) > 200 and abs(row.diffpred) < .01 and ((row.odiffpred>0) == (row.allelicRatio>.5)) :
        start=0
        end=256
        print(index)
        
        orig_seq = ctcfgen.encode(np.fromstring(genome[row.chr][row.start:row.end].lower(), dtype=np.uint8))
        new_seq = orig_seq.copy()
        new_seq[128:129] = ctcfgen.encode_string(row['alt'].lower())
        
        print('Sequence')
        viz_sequence.plot_icweights(orig_seq[start:end], colors=colors)
        print('Pred Difference ' + str(row.diffpred))
        print('Strength Pred Difference: ' + str(row.odiffpred))
        print('Score Difference ' + str(row.diffscore))
        print('RC Difference ' + str(row.allelicRatio))
        
        print('Mutation')
        viz_sequence.plot_weights((np.subtract(orig_seq,new_seq)*new_seq)[start:end], colors=colors)
        fseq, orig_score = ctcfgen.get_pwm([orig_seq], get_score=True)
        print('Original sequence weighting:')
        get_importance(orig_seq, start=start, end=end)
        get_importance(orig_seq, my_model=model2, start=start, end=end)
        print('New Sequence weighting:')
        get_importance(new_seq, start=start, end=end)
        get_importance(new_seq, my_model=model2, start=start, end=end)
        print('PWM ' + str(orig_score))
        viz_sequence.plot_weights(fseq, colors=colors)
        viz_sequence.plot_weights(fseq[start:end], colors=colors)

In [ ]:
#new data set!
bed_two = '/home/kal/data/IBD.CTCFpeak.bed'
ibd = pd.read_table(bed_two)
ibd.columns = 'HD chr region..mb. signal credible_nSNP tier2 trait trait.1 trait.reassigned sharingBF variant OR_CD OR_UC p_multi position A0 A1 AF_1KG info type certainty evidence.var evidence.set status P_mean_95 Func.refGene Gene.refGene GeneDetail.refGene ExonicFunc.refGene AAChange.refGene Coding TFBS epigenetic eQTL Disease CredibleSet CodingHuang SpliceSite Promoter pvalue TagSNP BestSNP PosteriorProb SeqA0 SeqA1'.split()

In [ ]:
for index, row in ibd.iterrows():
    if index == 30:
        print(str(row))

In [ ]:
get_act = K.function([model.input, K.learning_phase()], [layer_dict['bias'].output])


orig_preds = list()
new_preds = list()
o_preds = list()
n_preds = list()
orig_pwms = list()
new_pwms = list()
in_pwm = list()

for index, row in ibd.iterrows():
    
    orig_seq = ctcfgen.encode_string(row['SeqA0'].lower())
    orig_seq = orig_seq[:256]
    new_seq = ctcfgen.encode_string(row['SeqA1'].lower())
    new_seq = new_seq[:256]
    
    orig_pred = get_act([np.asarray([orig_seq]*32), 0])[0][0][0]
    new_pred = get_act([np.asarray([new_seq]*32), 0])[0][0][0]
    orig_preds.append(orig_pred)
    new_preds.append(new_pred)
    
    o_preds.append(model2.predict(np.asarray([orig_seq]*32))[0][0])
    n_preds.append(model2.predict(np.asarray([new_seq]*32))[0][0])
    
    fseq, orig_score = ctcfgen.get_pwm([orig_seq], get_score=True)
    fseq, new_score = ctcfgen.get_pwm([new_seq], get_score=True)   
    orig_pwms.append(orig_score)    
    new_pwms.append(new_score)
    
    in_pwm.append(np.amax((np.subtract(orig_seq,new_seq)*new_seq)*fseq)>0)

In [ ]:
ibd['refpred'] = np.asarray(orig_preds)
ibd['altpred'] = np.asarray(new_preds)

ibd['refscore'] = np.asarray(orig_pwms)
ibd['altscore'] = np.asarray(new_pwms)

ibd['diffpred'] = ibd['altpred']-ibd['refpred']
ibd['diffscore'] = ibd['altscore']-ibd['refscore']

ibd['odiffpred'] = np.asarray(n_preds) - np.asarray(o_preds)

ibd['pwm_motif'] = np.asarray(in_pwm)

ibd.to_csv('/home/kal/data/IBD.CTCFpeaks.ml.bed', sep='\t', index=False, header=False)

In [ ]:
def mutagenisis(seq, model=model):
    # do a mutant scan
    mutant_window=1

    # get a mutant batch generator
    mutant_gen = ctcfgen.ngram_mutant_gen(seq, n=mutant_window)
    g = ctcfgen.filled_batch(mutant_gen, batch_size=batch_size)
    
    # base importances as large-step gradients
    # score with base there - average of scores without base there
    mutant_preds = list()
    for batch in g:
        mutant_preds.append(model.predict_on_batch(batch))

    return np.asarray(mutant_preds).reshape((-1, 4))  

In [ ]:
for index, row in ibd.iterrows():
    if row.odiffpred != 0 or row.diffpred != 0:
        start=0
        end=256
        print(index)
        
        orig_seq = ctcfgen.encode_string(row['SeqA0'].lower())
        orig_seq = orig_seq[:256]
        new_seq = ctcfgen.encode_string(row['SeqA1'].lower())
        new_seq = new_seq[:256]
        
        print('Sequence')
        viz_sequence.plot_icweights(orig_seq[start:end], colors=colors)
        print('Sigmoid Difference ' + str(row.diffpred))
        print('Tanh difference ' + str(row.odiffpred))
        print('Score Difference ' + str(row.diffscore))
        
        print('Mutation')
        viz_sequence.plot_weights((np.subtract(orig_seq,new_seq)*new_seq)[start:end], colors=colors)
        fseq, orig_score = ctcfgen.get_pwm([orig_seq], get_score=True)
        print('Sigmoid')
        get_importance(orig_seq, start=start, end=end)
        get_importance(new_seq, start=start, end=end)
        print('Tanh')
        get_importance(orig_seq, model=model2, start=start, end=end)
        get_importance(new_seq, model=model2, start=start, end=end)
        print('PWM ' + str(orig_score))
        viz_sequence.plot_weights(fseq, colors=colors)
        viz_sequence.plot_weights(fseq[start:end], colors=colors)

In [ ]:
# process the memes
def process_meme(meme_path):
    with open(meme_path, 'r') as infile:
        meme_length = -1
        memes = list()
        for line in infile.readlines():
            if 'letter-probability matrix' in line:
                meme_length = int(line.split()[5])
                this_meme_lines = list()
            elif meme_length > 0:
                this_meme_lines.append([float(item.strip()) for item in line.split()])
                meme_length = meme_length - 1
            elif meme_length == 0:
                this_meme = np.asarray(this_meme_lines)
                memes.append(this_meme)
                meme_length = -1
        if meme_length == 0:
            this_meme = np.asarray(this_meme_lines)
            memes.append(this_meme)
            meme_length = -1
        
    # add rcs of memes
    rcs = list()
    for meme in memes:
        rcs.append(meme[::-1, ::-1])
    memes = memes + rcs

    
    psuedocount=0.25
    # get the transformed memes
    transformed_memes = list()
    for meme in memes:
        meme = meme + psuedocount 
        norms = np.repeat(np.linalg.norm(meme, axis=1), 4).reshape((-1, 4))
        meme = np.log(meme/norms)
        min = np.amin(meme)
        meme = meme - min
        transformed_memes.append(meme)
    
    return transformed_memes

CTCF_memes = process_meme('/home/kal/data/CTCF.meme')
mystery_memes = process_meme('/home/kal/CTCF/mystery_seq/data/mystery_motif/mystery_motif.meme')

viz_sequence.plot_weights(mystery_memes[0])

def get_pwm(input_seqs, meme=None, position=None, meme_library=CTCF_memes, get_score=True, get_loc=False):  
    # get position and meme if not specified
    if meme == None and position == None:
        get_loc = True
        for seq in input_seqs:
            best_score = -np.inf
            for test_meme in meme_library:
                correlations = correlate2d(seq, test_meme, mode='valid')
                if np.max(correlations) > best_score:
                    best_score = np.max(correlations)
                    best_location = np.argmax(correlations)
                    best_filter = test_meme
        position = best_location
        meme = best_filter
    else:
        get_loc = False
    
    # get the pwms
    output_scores = list()
    pwm = np.zeros(input_seqs[0].shape)
    pwm[position:position+meme.shape[0]] = meme
    for seq in input_seqs:
        output_scores.append(np.copy(pwm))  
    
    if get_score:
        corr = correlate2d(seq[position:position+meme.shape[0]], meme, mode='valid')
        return pwm, np.max(corr)
            
    if get_loc:
        return np.asarray(output_scores), meme, position
    else:
        return np.asarray(output_scores)